In [4]:
import warnings
import optuna

warnings.filterwarnings('ignore', category=UserWarning)

## Demo 1
Find the value of x that minimizes the function: f(x) = (x - 2)^2 + 1

In [ ]:
# Find the value of x that minimizes the function: f(x) = (x - 2)^2 + 1

def objective(trial):
    pass

# Create and run study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
print(f"{study.best_params = }")
print(f"{study.best_value = }")

## Exercise 1
Find the values of x and y that mininize the function: f(x,y) = (x - 2)^2 + (y + 3)^2 + 1

In [ ]:
def objective(trial):
    pass

# Create and run study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

## Visualization

In [ ]:
# Visualize the optimization process

# Get optimization history
optuna.visualization.plot_optimization_history(study)


In [ ]:
# You can also visualize parameter importance
optuna.visualization.plot_param_importances(study)


## Optimize LightGBM estimator

### Prepare Dataset

In [3]:
# Load and prepare data
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd


data = load_breast_cancer()
print(f"Dataset shape: {data.data.shape}")  # 569 samples, 30 features
print("\nTarget distribution:")
print(f"Malignant (0): {(data.target == 0).sum()}")
print(f"Benign (1): {(data.target == 1).sum()}")
print("\nFeatures:", data.feature_names.tolist())

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.2, random_state=42
)
feature_names = data.feature_names.tolist()
X_train = pd.DataFrame(X_train, columns=feature_names)
X_test = pd.DataFrame(X_test, columns=feature_names)

Dataset shape: (569, 30)

Target distribution:
Malignant (0): 212
Benign (1): 357

Features: ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score


def objective(trial):
    # Define the hyperparameters to optimize
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'num_leaves': trial.suggest_int('num_leaves', 20, 50),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'verbose': -1  # Add this to suppress LightGBM logger

    }
    
    # Create model with suggested hyperparameters
    model = LGBMClassifier(**params, random_state=42)
    
    # Use cross-validation to evaluate the model
    cv_scores = cross_val_score(
        model, 
        X_train, 
        y_train, 
        cv=5,
        scoring='accuracy'
    )
    
    # Return mean CV score
    return cv_scores.mean()  

# Create and run study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

In [ ]:
print(f"{study.best_params}")
print(f"{study.best_value * 100:.2f}%")

## Exercise 2:

In [ ]:
import optuna
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

scaler = StandardScaler()
# Using the same breast cancer dataset defined above
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate=0.1):
        super(SimpleNet, self).__init__()
        self.layer = nn.Linear(input_size, hidden_size)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.output = nn.Linear(hidden_size, 2)
        
    def forward(self, x):
        x = self.layer(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

def train_model(model, train_loader, criterion, optimizer, device='cpu'):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate_model(model, test_loader, device='cpu'):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
    return correct / total

def objective(trial):
    # Student Task 1: Define hyperparameters to optimize
    # Use trial.suggest_float, trial.suggest_int, etc.
    # Example format:
    # hidden_size = trial.suggest_int('hidden_size', min_value, max_value)
    # learning_rate = 
    # dropout_rate = 
    # batch_size = 
    # n_epochs = 
    
    # Create model and move to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SimpleNet(input_size=20, 
                     hidden_size=hidden_size, 
                     dropout_rate=dropout_rate).to(device)
    
    # Create data loaders
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(n_epochs):
        train_loss = train_model(model, train_loader, criterion, optimizer, device)
        accuracy = evaluate_model(model, test_loader, device)
        
        # Enable pruning (early stopping) based on accuracy
        trial.report(accuracy, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return accuracy



In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, timeout=60)

# TODO:
# Print best trial information
# Create visualizations

## FLAML Demo

In [6]:

from sklearn.metrics import accuracy_score, roc_auc_score
from flaml import AutoML


# Initialize the AutoML object
automl = AutoML()

# Configure and train the model
# We'll specify multiple estimators for FLAML to try
automl.fit(
    X_train, 
    y_train,
    task="classification",  # Specify the task (classification in this case)
    time_budget=60,  # Time budget in seconds
    metric="accuracy",  # Optimization metric
    # List of estimators to try
    estimator_list=["lgbm", "rf", "xgboost", "extra_tree", "catboost"],  
    verbose=2  # Verbosity level
)

# Get the best model
best_model = automl.model

# Print information about the best model found
print("\n----- Best Model Information -----")
print(f"Best model: {automl.best_estimator}")
print(f"Best configuration: {automl.best_config}")
print(f"Training accuracy: {automl.best_loss * -1 if automl.best_loss < 0 else automl.best_loss}")
print(f"Time taken: {automl.best_config_train_time:.2f} seconds")

# Make predictions with the best model
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Evaluate on test data
test_accuracy = accuracy_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred_proba)

print("\n----- Test Performance -----")
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test AUC: {test_auc:.4f}")

# Summarize all models FLAML tried, ordered from best to worst
print("\n----- All Models Evaluated -----")
for i, (estimator, config, loss, train_time) in enumerate(
    zip(
        automl.estimator_list, 
        automl.config_list, 
        automl.loss_list, 
        automl.time_list
    )
):
    print(f"{i+1}. {estimator} - Accuracy: {-loss if loss < 0 else loss:.4f}, Time: {train_time:.2f}s")

ImportError: cannot import name 'AutoML' from 'flaml' (/Users/somto/Documents/projects/tutorial-seven/.venv/lib/python3.12/site-packages/flaml/__init__.py)